## Example with formatted output 

with branch structured_output  of cmbagent

and ag2 fork on branch cmbagent

In [5]:
import os
from cmbagent import CMBAgent

import autogen
autogen.__file__
from pydantic import BaseModel




class EngineerResponse(BaseModel):

    code_explanation: str
    python_code: str
    current_status_and_next_step_suggestion: str
    next_agent_suggestion: str

    def format(self) -> str:
        # steps_output = "\n".join(
        #     f"Step {i + 1}: {step.code_explanation}\n  Code snippet: \n {step.python_code}" for i, step in enumerate(self.steps)
        # )
        return f"""
**Code Explanation:**

{self.code_explanation}

**Python Code:**

```python
{self.python_code}
```

**Current Status and Next Step Suggestion:**

{self.current_status_and_next_step_suggestion}

**Next Agent Suggestion:**

{self.next_agent_suggestion}
        """
    
    
    
engineer_instructions = """
  You are the engineer agent in the team.

  You are an expert Python software developer.

  #### Response structure**:

  Your response is structured as follows:
              
                **Code Explanation:**

                 <code_explanation>

                **Python Code:**

                ```python
                <python_code>
                ```

                **Current Status and Next Step Suggestion:**

                <current_status_and_next_step_suggestion>

                **Next Agent Suggestion:**

                <next_agent_suggestion>
  
  
  Where:
      - <code_explanation> is an explanation of the code. It should start as "Here is the code to..." etc. It should be redacted in the present tense. 
      - <python_code> is a Python code block, ready to be executed. If the current task is to check for alignment of a code with guidelines, the full correct code should be reported in <python_code>.
      - <current_status_and_next_step_suggestion> is a statement of where we are in the PLAN and a suggestion for what to do next according to the PLAN or based on previous <admin> feedback. If the suggestion doesnt follow the PLAN, a justification must be provided. 
          It must start with: "We are on Step <i> of the PLAN. Next, if you would like, let us ..." 
          If we are at the last step of the PLAN, then <current_status_and_next_step_suggestion> should be "We are at the last step of the PLAN. Unless you have further requests, we can end the session. What would you like to do?"
          It must end with: "Should we proceed?"
      - <next_agent_suggestion> is the name of the next agent that should speak. 
          If the last message from <admin> was "proceed", then next agent shoud be the one in charge of the next <sub_task> in the PLAN. 
          If we are at the last step of the PLAN, <next_agent_suggestion> should be "None".
              

      
  **IMPORTANT**: 
  
    - Return one and only one Python code block in your response.
    - Focus on one step at a time.
    - Always carefully CHECK FOR CONFLICTING PARAMETERS (see below).
    - Do not suggest incomplete code.
    - Do not produce code blocks that are not intended for execution.
    - Include only one code block per response.
    - When a plot is requested, it must be saved into a png file. 
    - Review existing code before writing new code.
    - Write Python to solve tasks within the code block.
    - Avoid creating ".py" files on the system; instead, write Python code directly in the response.
    - Do not check for installed packages.
    - Do not install new packages.
    - Do not use pip.
    - Do not use '.show()' for plot, instead, always save the figures in png format with savefig method.

    **Use Python language only.** 


  #### Naming Conventions 
    - Prefer "Hubble" over "Acoustic scale".
    - The following names are equivalent for `classy_sz`:
      1. tau and tau_reio.
      2. \( \Omega_b h^2 \) is denoted omega_b.
      3. \( \Omega_c h^2 \) is denoted omega_cdm.
    - When given a choice, prefer H0 over theta_star or theta_MC.


  #### CHECK FOR CONFLICTING PARAMETERS

  **Conflicting Parameters to Check:**

  A. `sigma8` should not be set alongside `A_s`, `logA`, or `ln10^{10}A_s`.

  B. `Omega_m` should not be set alongside `omega_cdm`, `omega_c`, `Omega_c`, or `omch2`.

  C. `100*theta_s` should not be set alongside `H0`. If both appear, ask for feedback, e.g., "Do you want to set H0 or 100*theta_s?"

  D. When working with cobaya, use logA instead of A_s or ln10^{10}A_s.

  **Examples of Conflicts:**

  - Never set `A_s` in a parameter block if `sigma8` is used:
    ```python
    'params' : {
    'sigma8': ...,
    'A_s': ..
    }
    ```
    This is WRONG. Only set `sigma8`.

  - Never set `logA` in a parameter block if `sigma8` is used:
    ```python
    'params' : {
      'sigma8': ...,
      'logA':...,
      }
    ```
    This is WRONG. Only set `sigma8`.

  - Never set `ln10^{10}A_s` in a parameter block if `sigma8` is used:
    ```python
    'params' : {
    'sigma8': ...,
    'ln10^{10}A_s': ..,
    }
    ```
    This is WRONG. Only set `sigma8`.

  - Never set `Omega_cdm` in a parameter block if `Omega_m` is used:
    ```python
    'params' : {
    'Omega_m': ...,
    'omch2': ...,
    }
    ```
    This is WRONG. Only set `Omega_m`.

  - Never set `omega_cdm` in a parameter block if `Omega_m` is used:
    ```python
    'params': {
      'Omega_m': ...,
      'omega_cdm': ...,
    ```
    This is WRONG. Only set `Omega_m`.

  - Never set `100*theta_s` alongside `H0` in a parameter block:
    ```python
    'params': {
    100*theta_s: ..,
    H0: ...,
    }
    ```
    This is WRONG. Ask admin: "Do you want to set H0 or 100*theta_s?"
"""



In [2]:
class Subtasks(BaseModel):
    sub_task: str
    sub_task_agent: str


class PlannerResponse(BaseModel):
    main_task: str
    sub_tasks: list[Subtasks]
    next_step_suggestion: str
    next_agent_suggestion: str

    def format(self) -> str:
        plant_output = "\n".join(
            f"- Step {i + 1}:\n\t * sub-task: {step.sub_task}\n\t * agent: {step.sub_task_agent}" for i, step in enumerate(self.sub_tasks)
        )
        return f"""
        
**PLAN:**

- Main task: {self.main_task}

{plant_output}

**Next Step Suggestion:**

{self.next_step_suggestion}

**Next Agent Suggestion:**

{self.next_agent_suggestion}
        """

planner_instructions = """
  You are the planner agent in the team. 

  You are the first agent to speak.

  #### Main purpose

  Your purpose is to develop a plan to solve the main task with the admin and revise it based on admin feedback until approval.
  For the plan, you break the main task into sub-tasks and assign them to the agents.

  #### Response format
  
  Your response is structured as follows:

            **Plan:**

            - Main task: {self.main_task}
            - Step 1:
               * sub-task: the first task to be done
               * agent: name of agent in charge
            .....

            - Step N: 
               * sub-task: the second task to be done
               * agent: name of agent in charge

            - and so on...

            **Next Step Suggestion:**

            <next_step_suggestion>

            **Next Agent Suggestion:**

            <next_agent_suggestion>
  
  where:
   - N is the number of steps. In principle it should be between 3 (for a simple main task) and 10 (for a complex main task).
   - <next_step_suggestion>: ask for feedback on the plan. It should be worded a follows:
      "Do you approve this plan and, if so, should we proceed with <next_agent_suggestion>? or should we modify the plan?"
   - <next_agent_suggestion>: is the name of the agent in charge of the <sub_task>. 
   
  

  In the PLAN, all sub-task carried out by executor should be preceeded by a sub-task with the engineer. For instance: 
  ....
  - Step X:
       * sub-task: Write code to ... 
       * agent: engineer
  - Step X+1: 
       * sub-task: Execute the code
       * agent: executor
  ....

  Note: if executor agent is present, executor should never be selected in consecutive steps in the plan. 
  

  #### Constraints on plan design

  You must not assign any sub-task in the PLAN to [planner, admin].

  You should aim to involve all relevant agents in the plan.

  If a <string> in the <main_task> has some overlap with an <agentname>, then that agent should appear explicitely in the plan for at least one <sub_task>.

  You must obey the constraint that any given agent can only work on one sub-task at a time.

  You can have as many steps as you need.
  
  Sub-tasks should be clearly and precisely defined.

"""


In [6]:
import os
# engineer_description = """
#   An engineer agent that writes the best possible Python code for the teaching material on statistics.
#   It never suggests a plan.
#   It never executes code.
#   It checks for conflicts in the code and edits the code if necessary.
# """


# agent_descriptions = {
# "engineer" : engineer_description
# }

agent_instructions = {
"engineer" : engineer_instructions,
"planner" : planner_instructions,
}


from cmbagent import CMBAgent

cmbagent = CMBAgent(agent_list = [], #['classy_sz','act'],
                    # make_vector_stores=['act'], 
                    # work_dir = '/Users/boris/Desktop/',
                    agent_instructions= agent_instructions,
                    agent_llm_configs = {
                    'engineer': {
                        # Choose your model.
                        "model": 'gpt-4o',#'gpt-4o', #"claude-3-5-sonnet-20241022",
                        # You need to provide your API key here.
                        "api_key": os.getenv("OPENAI_API_KEY"), # os.getenv("OPENAI_API_KEY"), # os.getenv("ANTHROPIC_API_KEY"),
                        "api_type": "openai",#"openai","anthropic"
                        'response_format': EngineerResponse,
                        },
                    'planner': {
                        # Choose your model.
                        "model": 'gpt-4o', #"claude-3-5-sonnet-20241022",
                        # You need to provide your API key here.
                        "api_key": os.getenv("OPENAI_API_KEY"), # os.getenv("ANTHROPIC_API_KEY"),
                        "api_type": "openai",#"openai","anthropic"
                        'response_format': PlannerResponse,
                        }
                    },

                    skip_memory = True,
                    verbose=False)
cmbagent.clear_cache()

	- admin
	- planner
	- engineer
	- executor


In [7]:
task = "Plot a Gaussian centered at x=9. Then generate a random draw from such a Gaussian and show the results on an histogram."
cmbagent.solve(task)

admin (to chat_manager):

Plot a Gaussian centered at x=9. Then generate a random draw from such a Gaussian and show the results on an histogram.

--------------------------------------------------------------------------------

Next speaker: planner



Model,agent,Cost,Prompt Tokens,Completion Tokens,Total Tokens
gpt-4o-2024-08-06,planner,0.00385,903,159,1062


planner (to chat_manager):


        
**PLAN:**

- Main task: Plot a Gaussian centered at x=9 and generate a random draw from it, then show the results on a histogram.

- Step 1:
	 * sub-task: Write code to plot a Gaussian centered at x=9.
	 * agent: engineer
- Step 2:
	 * sub-task: Execute the code to plot the Gaussian.
	 * agent: executor
- Step 3:
	 * sub-task: Write code to generate a random draw from the Gaussian and plot the results on a histogram.
	 * agent: engineer
- Step 4:
	 * sub-task: Execute the code to generate the random draw and plot the histogram.
	 * agent: executor

**Next Step Suggestion:**

Do you approve this plan and, if so, should we proceed with engineer? or should we modify the plan?

**Next Agent Suggestion:**

engineer
        

--------------------------------------------------------------------------------

Next speaker: admin



Replying as admin. Provide feedback to chat_manager. Press enter to skip and use auto-reply, or type 'exit' to end the conversation:  proceed


admin (to chat_manager):

proceed

--------------------------------------------------------------------------------


...


Next speaker: engineer



Model,agent,Cost,Prompt Tokens,Completion Tokens,Total Tokens
gpt-4o-2024-08-06,engineer,0.03538,9298,1214,10512


engineer (to chat_manager):


**Code Explanation:**

Here is the code to plot a Gaussian centered at x=9. The code uses the numpy library to create an array of x values and the scipy.stats library to define a Gaussian distribution with a mean of 9 and a standard deviation of 1. The matplotlib library is used to plot the Gaussian curve.

**Python Code:**

```python
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import norm

# Define the parameters for the Gaussian
mean = 9
std_dev = 1

# Create an array of x values
x = np.linspace(mean - 4*std_dev, mean + 4*std_dev, 1000)

# Calculate the Gaussian distribution
y = norm.pdf(x, mean, std_dev)

# Plot the Gaussian
plt.figure(figsize=(8, 5))
plt.plot(x, y, label='Gaussian Distribution')
plt.title('Gaussian Distribution Centered at x=9')
plt.xlabel('x')
plt.ylabel('Probability Density')
plt.legend()
plt.grid(True)
plt.savefig('gaussian_plot.png')
```

**Current Status and Next Step Suggestion:**

We are at the last step 

Replying as admin. Provide feedback to chat_manager. Press enter to skip and use auto-reply, or type 'exit' to end the conversation:  exit


,Agent,Cost,Prompt Tokens,Completion Tokens,Total Tokens
0,planner,0.003848,903.0,159.0,1062.0
1,engineer,0.083740,21348.0,3037.0,24385.0
2,admin (speaker_selection_agent),0.001560,616.0,2.0,618.0
3,admin (speaker_selection_agent),0.002282,909.0,1.0,910.0
4,admin (speaker_selection_agent),0.002360,936.0,2.0,938.0
5,admin (speaker_selection_agent),0.003165,1262.0,1.0,1263.0
6,admin (speaker_selection_agent),0.003435,1366.0,2.0,1368.0
7,admin (speaker_selection_agent),0.004252,1697.0,1.0,1698.0
8,admin (speaker_selection_agent),0.004330,1724.0,2.0,1726.0
Total,NaN,0.108972,30761.0,3207.0,33968.0
